In [7]:
from models.blipv2_ft import blip

In [8]:
import yaml

# config = yaml.load(open("configs/pretrain_blip.yaml", "r"), Loader=yaml.Loader)
config = yaml.load(open("configs/finetune.yaml", "r"), Loader=yaml.Loader)

In [9]:
device = "cuda"

model = blip(config=config)
model.to(device)

Resizing position embedding grid-size from %s to %s (16, 16) (28, 28)
_IncompatibleKeys(missing_keys=[], unexpected_keys=['visual.proj', 'visual.ln_post.weight', 'visual.ln_post.bias', 'visual.transformer.resblocks.23.attn.in_proj_weight', 'visual.transformer.resblocks.23.attn.in_proj_bias', 'visual.transformer.resblocks.23.attn.out_proj.weight', 'visual.transformer.resblocks.23.attn.out_proj.bias', 'visual.transformer.resblocks.23.ln_1.weight', 'visual.transformer.resblocks.23.ln_1.bias', 'visual.transformer.resblocks.23.mlp.c_fc.weight', 'visual.transformer.resblocks.23.mlp.c_fc.bias', 'visual.transformer.resblocks.23.mlp.c_proj.weight', 'visual.transformer.resblocks.23.mlp.c_proj.bias', 'visual.transformer.resblocks.23.ln_2.weight', 'visual.transformer.resblocks.23.ln_2.bias'])
load checkpoint from pretrain_blipv2/checkpoint_14.pth
_IncompatibleKeys(missing_keys=['visual_encoder.positional_embedding'], unexpected_keys=[])


BLIP(
  (visual_encoder): VisionTransformer(
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (drop_path): Identity()
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_fea

In [10]:
import torch

checkpoint_path = "pretrain_blipv2/checkpoint_best.pth"

state_dict = torch.load(checkpoint_path, map_location="cpu")["model"]

In [11]:
msg = model.load_state_dict(state_dict)

In [12]:
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode

# from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

from constant import OPENAI_DATASET_MEAN, OPENAI_DATASET_STD
transform_test = transforms.Compose([
    transforms.Resize(config['image_size'],interpolation=InterpolationMode.BICUBIC),
    transforms.CenterCrop(config['image_size']),
    transforms.ToTensor(),
    transforms.Normalize(OPENAI_DATASET_MEAN, OPENAI_DATASET_STD),
    ])       

In [16]:
from PIL import Image

image_path = "/export/home/workspace/dreambooth/diffusers/data/yb-cat/WechatIMG114.jpeg"

image = Image.open(image_path).convert("RGB")
image = transform_test(image).unsqueeze(0).to(device)


In [17]:
model.to(device)

BLIP(
  (visual_encoder): VisionTransformer(
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (drop_path): Identity()
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_fea

In [18]:
model.generate(image=image)

['a cat sitting on a table wearing an orange scarf']